# Process Named Entity Recognition on Article Text

### Imports

In [1]:
!which jupyter
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm import tqdm
import re

# NER Imports
import spacy
import en_core_web_sm
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Set up ner processor from SpaCy
ner_processor = en_core_web_sm.load()

/home/ubuntu/thesis_env2/bin/jupyter


In [2]:
df = pd.read_csv('./data/covid19_articles_20201231.csv')

In [3]:
# date to datetime
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369047 entries, 0 to 369046
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   author      181799 non-null  object        
 1   date        369047 non-null  datetime64[ns]
 2   domain      369047 non-null  object        
 3   title       368962 non-null  object        
 4   url         369047 non-null  object        
 5   content     369047 non-null  object        
 6   topic_area  369047 non-null  object        
dtypes: datetime64[ns](1), object(6)
memory usage: 19.7+ MB


In [4]:
df.head(3)

,author,date,domain,title,url,content,topic_area
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business


### NER Pre Processing & Tagging Functions

In [5]:
# Remove text from articles

def clean_text(df):
    search_text = "follow us on Facebook, Twitter, and Instagram" #2423
    print(len(df[df['content'].str.contains(search_text)]))
    df['content'] = df['content'].str.replace(search_text, "")
    search_text = "Follow Yahoo Finance on Twitter, Facebook, Instagram, Flipboard, LinkedIn, and reddit"
    print(len(df[df['content'].str.contains(search_text)]))
    df['content'] = df['content'].str.replace(search_text, "")
    search_text = "find us on Facebook and follow us on Twitter"
    print(len(df[df['content'].str.contains(search_text)]))
    df['content'] = df['content'].str.replace(search_text, "")
    search_text = "on Twitter, Facebook, LinkedIn, Instagram and YouTube"
    print(len(df[df['content'].str.contains(search_text)]))
    df['content'] = df['content'].str.replace(search_text, "")
    search_text = "Twitter - Facebook - Instagram"
    print(len(df[df['content'].str.contains(search_text)]))
    df['content'] = df['content'].str.replace(search_text, "")
    search_text = "earnings conference call | transcript"
    print(len(df[df['title'].str.contains(search_text, na=False)]))
    df['content'] = df['title'].str.replace(search_text, "")
    return df

# Drop long articles - cannot be processed by Spacy - articles where text longer than 1,000,000 characters

def drop_long_articles(df):
    num_articles_pre_drop = len(df)
    print('no. of articles: ',len(df))
    df = df[df['content'].apply(lambda x: len(x) <= 1000000)]
    num_articles_post_drop = len(df)
    print(f'no. of articles after dropping long articles: {len(df)}, no. articles dropped: {num_articles_pre_drop - num_articles_post_drop}') 
    return df

# NER Tagging

def ner_tagging(df, row_index_start):
    df_ner = df.copy()
    col_index = df_ner.columns.get_loc('org_names')

    for row in tqdm(range(len(df_ner.iloc[row_index_start:]))):
        row = row + row_index_start
        pos_tagged = ner_processor(df_ner.iloc[row]['content'])
        org_names = set([X.text for X in pos_tagged.ents if X.label_ == 'ORG'])
        df_ner.iat[row, col_index] = sorted(org_names)

        # Save every 10,000 articles processed
        if row % 10000 == 0:
            df_ner.to_pickle('./data/df_ner_210912.pickle')

    # Save df after tagging
    df_ner.to_pickle('./data/df_ner_210912.pickle')
    
    return df_ner

### Tag Full Data Set

In [6]:
# NER Tagging - takes about 15 hours for full dataset

tag_full_data = False
part_processing = False

if tag_full_data:
    
    if part_processing:
        df_ner = pd.read_pickle('./data/df_ner_210717.pickle') # Load part processed file
        row_label_start = df_ner[df_ner['org_names'] == ""].index[0]
        row_index_start = df_ner.index.get_loc(row_label_start) # Start point for continued processing
        df_ner = ner_tagging(df_ner, row_index_start) # For part processed file
        
    else:
        row_index_start = 0
        df = clean_text(df)
        df = drop_long_articles(df)
        df_ner = df.copy()
        df_ner['org_names'] = ''
        df_ner = ner_tagging(df_ner, row_index_start)
        
    df_ner.to_pickle('./data/df_ner_210912.pickle')

In [7]:
#df_ner = pd.read_pickle('./data/df_ner_210717.pickle')
#df_ner = pd.read_pickle('./data/df_ner.pickle')
df_ner = pd.read_pickle('./data/df_ner_210912.pickle')

In [8]:
df_ner

,author,date,domain,title,url,content,topic_area,org_names
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,"[BA, Boeing, CAT, CMI, Caterpillar, Cummins In..."
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business,"[ADP, AKA, CTAS, Cintas, Cloud-Based Services ..."
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business,"[Credit Suisse, FactSet, Ford, NYSE, Tesla, We..."
3,Roberto Torres,2020-01-03,ciodive,"On the road to AI adoption, execs grapple with...",https://www.ciodive.com/news/ai-adoption-execs...,CIOs kicked off 2019 with AI as an item to wat...,tech,"[AI, CIO, Gartner, Gartner's, H&M, IBM, IBM Data]"
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer,"[11:30pm PST, BAFTA, CDC, Chanel, Instagram, L..."
...,...,...,...,...,...,...,...,...
369042,Polly Harrison,2020-12-31,thefintechtimes,A Human Touch Will Be a Competitive Edge After...,https://thefintechtimes.com/53867-2/,Niels Pedersen is a Chartered Accountant and S...,finance,"[ATM, BBC Radio Scotland, C. Hoare & Co., Lloy..."
369043,NaN,2020-12-31,marketscreener,Datametrex AI : Announces Deploying NexaSecuri...,https://www.marketscreener.com/quote/stock/DAT...,Datametrex AI Limited (TSXV: DM) (FSE: D4G) (O...,business,"[Artificial Intelligence, Artificial Intellige..."
369044,Polly Harrison,2020-12-31,thefintechtimes,"US Payments: Smart Pension, Episode Six, PAAY ...",https://thefintechtimes.com/us-payments-smart-...,"This December, The Fintech Times is asking ind...",finance,"[EMV, Jodan Ledford, PAAY, PEP, PR & Editorial..."
369045,NaN,2020-12-31,marketscreener,"WESTWATER RESOURCES, INC. : Entry into a Mater...",https://www.marketscreener.com/quote/stock/WES...,Item 1.01Entry into a Material Definitive Agre...,business,"[Energy\nCorp., Glimpses, Neutron Energy, Inc...."


In [6]:
df_ner[261631:261661]

,author,date,domain,title,url,content,topic_area,org_names
261633,NaN,2020-08-05,marketscreener,CoreCivic : Corporate Structure Presentation,https://www.marketscreener.com/quote/stock/COR...,C-Corp Conversion & Capital Allocation Strateg...,business,"[ADJUSTED EBITDA, ADOC ALABAMA FACILITY OVERVI..."
261634,NaN,2020-08-05,marketscreener,ALCONIX : Summary of Consolidated Financial Re...,https://www.marketscreener.com/quote/stock/ALC...,"August 6, 2020 Summary of Consolidated Financi...",business,"[AI, ALCONIX, Additional Information Effect of..."
261635,NaN,2020-08-05,marketscreener,Victory Capital : Reports Second Quarter 2020 ...,https://www.marketscreener.com/quote/stock/VIC...,Second Quarter 2020 Highlights Victory Capital...,business,"[2The Company, AUM, About Victory Capital Vict..."
261636,NaN,2020-08-05,marketscreener,Darling Ingredients Inc. : Reports Second Quar...,https://www.marketscreener.com/quote/stock/DAR...,"IRVING, Texas, Aug. 5, 2020 /PRNewswire/ -- Da...",business,"[ASF, Adjusted EBITDA, BSE, Company, Consolida..."
261637,NaN,2020-08-05,marketscreener,Summit Hotel Properties : Reports Second Quart...,https://www.marketscreener.com/quote/stock/SUM...,"AUSTIN, Texas, Aug. 5, 2020 /PRNewswire/ -- Su...",business,"[91.54 Actual Statistics Rooms, ADR, AFFO, Adj..."
261638,NaN,2020-08-05,marketscreener,Essential Properties Realty Trust : Announces ...,https://www.marketscreener.com/quote/stock/ESS...,- Collected 87% of July Rent - - Second Quarte...,business,"[ABR, AFFO, AFFO The Company, ASC 326, ATM Pro..."
261639,NaN,2020-08-05,marketscreener,CVS Health : Reports Second Quarter Results; D...,https://www.marketscreener.com/quote/stock/CVS...,"WOONSOCKET, R.I., Aug. 5, 2020 /PRNewswire/ --...",business,"[% Prescriptions, 512 Interest, Adjusted EPS, ..."
261640,NaN,2020-08-05,marketscreener,"Energizer Holdings, Inc. : Announces Fiscal 20...",https://www.marketscreener.com/quote/stock/ENE...,"ST. LOUIS, Aug. 5, 2020 /PRNewswire/ -- Energi...",business,"[106.9 99.9 Research, A&P, Acquisition, Adjust..."
261641,NaN,2020-08-05,marketscreener,DBS : Second-Quarter Net Income Fell 22% On Hi...,https://www.marketscreener.com/quote/stock/DBS...,By P.R. Venkat DBS Group Holdings Ltd.'s secon...,business,"[DBS, Factset, P.R. Venkat DBS Group Holdings ..."
261642,NaN,2020-08-05,marketscreener,Cerecor to Present at the 2020 Wedbush PacGrow...,https://www.marketscreener.com/quote/stock/CER...,"ROCKVILLE, Md., Aug. 05, 2020 (GLOBE NEWSWIR...",business,"[About Cerecor Cerecor, Adult Onset Stills Dis..."


### Tag Labelled File

In [32]:
tag_labelled_data = False

if tag_labelled_data:

    df_lab = pd.read_csv('./data/df_lab_latest.csv', index_col=0, converters={'org_names_listed': eval, 'org_names_listed_uo': eval, 'listing_country': eval})
    df_lab = clean_text(df_lab)
    df_lab = drop_long_articles(df_lab)
    df_lab_ner = ner_tagging(df_lab, 0)
    
    df_lab_ner.to_pickle('./data/df_lab_ner.pickle')
    df_lab_ner.to_csv('./data/df_lab_ner.csv')

else:
    df_lab_ner = pd.read_pickle('./data/df_lab_ner.pickle')

In [8]:
df_lab_ner.head(6)

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo,listed_count,listed_uo_count,listing_country,listing_country_count,org_names
272801,NaN,2020-08-15,finance.yahoo,Julen Lopetegui: We must be at our best to bea...,https://finance.yahoo.com/news/julen-lopetegui...,Head coach Julen Lopetegui knows Sevilla will ...,business,[],[],0,0,[],0,"[Inter Milan, Jose Mourinho’s, Lopetegui, Manc..."
200452,NaN,2020-07-11,finance.yahoo,Gallagher Premiership teams will not be punish...,https://uk.finance.yahoo.com/news/gallagher-pr...,Gallagher Premiership clubs will have flexibil...,business,[],[],0,0,[],0,"[Childs, Premiership Rugby, the British & Iris..."
360615,NaN,2020-12-14,marketscreener,"Don't stockpile food, minister tells British s...",https://www.marketscreener.com/quote/currency/...,"Britain and the EU agreed on Sunday to ""go the...",business,[Tesco PLC],[Tesco],1,1,[United Kingdom],1,"[BBC, BRC, EU, Sharma, Sky News, Tesco, The Br..."
348240,Joe Hoppe,2020-11-23,marketwatch,"Wynnstay Properties profit halves, lifts dividend",https://www.marketwatch.com/story/wynnstay-pro...,Wynnstay Properties PLC said Monday that its f...,business,[Wynnstay Properties Plc],[Wynnstay Properties PLC],1,1,[United Kingdom],1,[Wynnstay Properties PLC]
150912,NaN,2020-06-11,finance.yahoo,U.K. Scientists Defy Johnson to Speak Out on V...,https://ca.finance.yahoo.com/news/johnson-unde...,(Bloomberg) -- U.K. Prime Minister Boris Johns...,business,[],[],0,0,[],0,"[Conservative Party, Dominic Cummings, Ferguso..."
5919,Barbara Kollmeyer,2020-02-18,marketwatch,Apple Warning Rattles European Tech Stocks,https://www.marketwatch.com/articles/apple-war...,AFP via Getty Images European stocks came unde...,business,"[Apple Inc., Dialog Semiconductor Plc, ASM Int...","[Apple, Dialog Semiconductor, ASM Internationa...",6,6,"[United States, United Kingdom, Netherlands, U...",6,"[AFP, ASM International, Apple, HSBC, Intesa S..."


### Clean Tagged Names Functions
- Remove leading and trailing non-letter characters e.g. ' % 2,4 ARCU Aircraft Leasing Limited' -> 'ARCU Aircraft Leasing Limited' Note: this would also drop 3M but companies with number as first letter are very rare
- Remove names that are only 1 or 2 letters
- Remove names found in org_stopwords list

In [8]:
# Load company name stopwords

def load_stopwords(file_path, sheet_name):    
    org_stopwords = pd.read_excel(filepath_stopwords, sheet_name = sheet_name, header = None)
    org_stopwords = list(set(sorted(org_stopwords[0])))
    new_stopwords = ['company']
    org_stopwords = org_stopwords + new_stopwords
    org_stopwords = sorted([word.lower() for word in org_stopwords])
    print('number of phrases in org stopwords list: ', len(org_stopwords))
    return org_stopwords

# Clean Tagged Names

def clean_tagged_names(df, org_stopwords):

    df_ner_clean = df.copy()
    col_index = df_ner_clean.columns.get_loc('org_names')

    for idx, org_names in tqdm(enumerate(df_ner_clean['org_names'])):

        # remove leading and trailing non-letter characters
        org_names = set([re.sub('^[^a-zA-Z]*|[^a-zA-Z]*$','',name) for name in org_names])

        # remove names that are less than 4 letters
        org_names = set([name.lower() for name in org_names if len(name) > 3])

        # exclude names in org_stopwords list - difference between 2 sets
        org_names = org_names - set(org_stopwords) 
        df_ner_clean.iat[idx, col_index] = sorted(org_names)
        
    return df_ner_clean

# Remove found names that are a substring of a longer name also found

def remove_substring_names(df):
    
    df = df.copy()
    col_index = df.columns.get_loc('org_names')

    for idx, org_names in tqdm(enumerate(df['org_names'])):
        #print(org_names)
        #print(type(org_names))
        org_names_new = []
        for name in org_names:
            #print('name is', name, name in org_names)
            org_names_check = set(org_names) - set([name])
            #print(org_names_check)
            if not any(name in string for string in org_names_check):
                org_names_new.append(name)
        #print('org_names_new', org_names_new)
        df.iat[idx, col_index] = sorted(org_names_new)
    
    return df

In [9]:
# Load stopwords
filepath_stopwords = './data/company_stopwords.xlsx'
sheet_name = 'all'

# Process labelled data set 
process_labelled_data = False

if process_labelled_data:
    org_stopwords = load_stopwords(filepath_stopwords, sheet_name)
    df_lab_ner = pd.read_pickle('./data/df_lab_ner.pickle')
    df_lab_ner_clean = clean_tagged_names(df_lab_ner, org_stopwords)
    df_lab_ner_clean = remove_substring_names(df_lab_ner_clean)
    df_lab_ner_clean.to_pickle('./data/df_lab_ner_clean.pickle')
else:
    df_lab_ner_clean = pd.read_pickle('./data/df_lab_ner_clean.pickle')
    
# Process full data set 
process_full_data = True

if process_full_data:
    org_stopwords = load_stopwords(filepath_stopwords, sheet_name)
    #df_ner = pd.read_pickle('./data/df_ner.pickle')
    df_ner = pd.read_pickle('./data/df_ner_210912.pickle')
    df_ner_clean = clean_tagged_names(df_ner, org_stopwords)
    df_ner_clean = remove_substring_names(df_ner)
    df_ner_clean.to_pickle('./data/df_ner_clean_210912.pickle')
else:
    #df_ner_clean = pd.read_pickle('./data/df_ner_clean.pickle')
    df_ner_clean = pd.read_pickle('./data/df_ner_210717.pickle')
    print(len(df_ner_clean))

number of phrases in org stopwords list:  141


369045it [00:21, 17455.69it/s]
369045it [00:30, 12108.26it/s]


In [10]:
df_lab_ner_clean.head(3)

,author,date,domain,title,url,content,topic_area,org_names_listed,org_names_listed_uo,listed_count,listed_uo_count,listing_country,listing_country_count,org_names
272801,NaN,2020-08-15,finance.yahoo,Julen Lopetegui: We must be at our best to bea...,https://finance.yahoo.com/news/julen-lopetegui...,Head coach Julen Lopetegui knows Sevilla will ...,business,[],[],0,0,[],0,"[inter milan, jose mourinho’s, lopetegui, roma..."
200452,NaN,2020-07-11,finance.yahoo,Gallagher Premiership teams will not be punish...,https://uk.finance.yahoo.com/news/gallagher-pr...,Gallagher Premiership clubs will have flexibil...,business,[],[],0,0,[],0,"[childs, premiership rugby, the british & iris..."
360615,NaN,2020-12-14,marketscreener,"Don't stockpile food, minister tells British s...",https://www.marketscreener.com/quote/currency/...,"Britain and the EU agreed on Sunday to ""go the...",business,[Tesco PLC],[Tesco],1,1,[United Kingdom],1,"[sharma, tesco]"


In [11]:
if process_full_data:
    df_ner_clean.head(3)

### General queries/checking on tagged names

In [13]:
import itertools 

# List of all org names
ner_org_names = sorted(set(list(itertools.chain(*[names for names in df_ner_clean['org_names']]))))
df_ner_org_names = pd.DataFrame(ner_org_names)
df_ner_org_names.to_pickle('./data/df_ner_org_names.pickle')
len(ner_org_names)

683575

In [14]:
#df_ner[df_ner['org_names'].apply(lambda x: 'guaranty agreement' in x)]
#df_ner_clean[df_ner_clean['org_names'].apply(lambda x: 'a\ncredit and guaranty agreement' in x)]

In [15]:
#ner_org_names[::-1]
ner_org_names[:5]

['" - Department of Economic Affairs',
 '" - Ministry of Finance',
 '" Arakan Army',
 '" BUSINESS REVIEW Office Software and',
 '" Banking & Payments Federation Ireland']

### Import listed companies

Import listed companies from pre compiled list and reduce to countries of interest

In [12]:
def load_listed_companies(file_path, sheet_name, countries):

    companies_list = pd.read_excel(file_path, sheet_name = sheet_name)

    # Drop companies names that are 2 or 3 characters long
    companies_list = companies_list.drop(labels = companies_list[companies_list['name'].str.len() < 4].index)

    # Countries of interest
    companies_list = companies_list[companies_list['country'].isin(countries)]
    
    return companies_list

In [13]:
# Import listed companies
filepath_companies = './data/company_names_listed.xlsx'
sheet_name = 'company_names'
countries_included = ['United States', 'Canada', 'Australia', 'United Kingdom']

companies_list = load_listed_companies(filepath_companies, sheet_name, countries_included)

print(companies_list['country'].value_counts())
print(len(companies_list))

United States     8848
Canada            3405
Australia         1873
United Kingdom    1424
Name: country, dtype: int64
15550


In [14]:
companies_list.head(3)

,id,name,common_names,ticker_symbol,country,industry,subindustry
0,35931,"The Ultimate Software Group, Inc.",NaN,ULTI,United States,Information Technology,Software
1,35908,"U.S. Personnel, Inc.",NaN,USPE,United States,Industrials,Professional Services
2,203152,e.Digital Corporation,NaN,EDIG,United States,Information Technology,"Technology Hardware, Storage and Peripherals"


### Most common words in listed companies names

Use in matching algorithm

In [15]:
from collections import Counter
results = Counter()
companies_list['name'].str.lower().str.split().apply(results.update)
companies_list_common_words = sorted([k for k, v in results.items() if v > 99])

In [16]:
#print(results)

In [17]:
print(companies_list_common_words)

['&', 'american', 'and', 'bancorp,', 'bank', 'capital', 'company', 'corp.', 'corporation', 'energy', 'energy,', 'exploration', 'financial', 'first', 'global', 'gold', 'group', 'group,', 'holdings', 'holdings,', 'inc.', 'incorporated', 'international', 'international,', 'investment', 'limited', 'ltd', 'ltd.', 'metals', 'minerals', 'mining', 'of', 'oil', 'pharmaceuticals,', 'plc', 'resources', 'services', 'systems,', 'technologies', 'technologies,', 'technology', 'the', 'therapeutics,', 'trust', 'ventures']


### Drop countries not interested in from labelled file

Labelled data file has listed companies from many countries, drop those countries not being considered

In [18]:
# Drop countries not interested in from labelled file

def drop_countries_labelled_file(df):
    
    df_lab_ner_clean = df.copy()
    
    col_index_lc = df_lab_ner_clean.columns.get_loc('listing_country')
    col_index_onl = df_lab_ner_clean.columns.get_loc('org_names_listed')
    col_index_onluo = df_lab_ner_clean.columns.get_loc('org_names_listed_uo')
    col_index_lc2 = df_lab_ner_clean.columns.get_loc('listed_count')

    for idx in range(len(df_lab_ner_clean)):
        org_names_listed = []
        org_names_listed_uo = []
        countries_to_add = []

        for idx_c, country in enumerate(df_lab_ner_clean.iloc[idx, col_index_lc]):
            if country in countries_included:
                listed_name = df_lab_ner_clean.iat[idx, col_index_onl][idx_c]
                listed_name_uo = df_lab_ner_clean.iat[idx, col_index_onluo][idx_c]
                org_names_listed.append(listed_name)
                org_names_listed_uo.append(listed_name_uo)
                countries_to_add.append(country)

        df_lab_ner_clean.iat[idx, col_index_onl] = org_names_listed
        df_lab_ner_clean.iat[idx, col_index_onluo] = org_names_listed_uo
        df_lab_ner_clean.iat[idx, col_index_lc] = countries_to_add
        df_lab_ner_clean.iat[idx, col_index_lc2] = len(org_names_listed)
    
    df_lab_ner_clean = df_lab_ner_clean.drop(['listed_uo_count', 'listing_country_count'], axis=1)
    
    return df_lab_ner_clean

In [19]:
# Process country drop
process_country_drop = True

if process_country_drop:
    df_lab_ner_clean = pd.read_pickle('./data/df_lab_ner_clean.pickle')
    df_lab_ner_clean = drop_countries_labelled_file(df_lab_ner_clean)

In [20]:
df_lab_ner_clean.iloc[-1]['org_names']

['astrazeneca azn',
 'bntx',
 'fauci',
 'johns hopkins university',
 'johnson & johnson jnj',
 'merck & co. merk',
 'the national institute of allergy and infectious diseases']

### Checking similarity of NER word against company names in company list

Rule 1 
- When the NER is only 1 word  
- The comparison is reduced to the set of common words  
- When the similarity ratio is higher than 90% it is accepted

Rule 2
- When the NER is more than 1 word
- When part of the NER exists in the stop words list e.g. 'Capital One Financial Corp', financial and corp are words in the stop words list
- When the similarity ratio is higher than 95% it is accepted

Rule 3
- Remaining words
- Similarity ratio on best matching substring being 90% and higher

In [21]:
def match_companies(df, file_name='df_ner_temp'):
    
    df = df.copy()
    df['filtered_names'] = ''
    df['filtered_names_match'] = ''
    col_index_fn = df.columns.get_loc('filtered_names')
    col_index_fnm = df.columns.get_loc('filtered_names_match')
    #print_on = True
    print_on = False

    for idx, names in tqdm(enumerate(df['org_names'])):
        if print_on:
            print('\n', idx, names, 'labels: ', df.iat[idx,col_index_onl])
        filtered_names = []
        filtered_names_match = []
        for name in names:

            # Rule 1 - NER name only 1 word -> set of common words  -> similarity ratio on set
            if len(name.split(' ')) == 1:
                processor = lambda x: set(x.lower().split()) - set(companies_list_common_words)
                processor_2 = lambda x: x.lower()
                best_match = process.extractOne(query=name, choices=companies_list['name'], processor=processor, scorer=fuzz.token_sort_ratio, score_cutoff=98)
                #best_match = process.extractOne(query=name, choices=companies_list['name'], processor=processor_2, scorer=fuzz.ratio, score_cutoff=75)
                if best_match:
                    if print_on:
                        print('rule1: ', name, '-', best_match, best_match[1])
                    filtered_names.append(name)
                    filtered_names_match.append(best_match[0])            

            # Rule 2 - NER name longer than 1 word plus part of name in stopwords list -> similarity ratio on set
            elif set(name.split(' ')).intersection(org_stopwords):
                processor_2 = lambda x: x.lower()
                best_match = process.extractOne(query=name, choices=companies_list['name'], processor=processor_2, scorer=fuzz.token_set_ratio, score_cutoff=95)
                if best_match:
                    if print_on:
                        print('rule2: ', name, '-', best_match[0], best_match[1])
                    filtered_names.append(name)
                    filtered_names_match.append(best_match[0])

            else:
                best_match = process.extractOne(query=name, choices=companies_list['name'], scorer=fuzz.partial_ratio, score_cutoff=96)
                if best_match:
                    if print_on:
                        print(f'best match: {best_match}')
                        print('rule3: ', name, '-', best_match[0], best_match[1])
                    filtered_names.append(name)
                    filtered_names_match.append(best_match[0])

        df.iat[idx,col_index_fn] = sorted(list(set(filtered_names)))
        df.iat[idx,col_index_fnm] = sorted(list(set(filtered_names_match)))
        
        #if idx % 5000 == 0:
         #   df.to_pickle('./data/' + file_name + '.pickle')
    
    return df

In [22]:
def check_matches(df):
    
    df['count_matches'] = ''
    df['count_lab_not_found'] = ''
    df['count_found_not_lab'] = ''
    col_index_onm = df.columns.get_loc('org_names_listed')
    col_index_fnm = df.columns.get_loc('filtered_names_match')
    col_index_cm = df.columns.get_loc('count_matches')
    col_index_clnf = df.columns.get_loc('count_lab_not_found')
    col_index_cfnl = df.columns.get_loc('count_found_not_lab')

    for idx in range(len(df)):
        # True Positives
        count_matches = len(list(set(df.iloc[idx,col_index_onm]).intersection(df.iloc[idx,col_index_fnm])))
        # False Negatives
        count_lab_not_found = len(list(set(df.iloc[idx,col_index_onm]).difference(df.iloc[idx,col_index_fnm]))) # labelled but not found in NER
        # False Positives
        count_found_not_lab = len(list(set(df.iloc[idx,col_index_fnm]).difference(df.iloc[idx,col_index_onm]))) # found in NER but not labelled

        df.iat[idx, col_index_cm] = count_matches
        df.iat[idx, col_index_clnf] = count_lab_not_found
        df.iat[idx, col_index_cfnl] = count_found_not_lab
    
    return df

In [23]:
# Calculate performance

def calculate_performance(df):

    true_pos = sum(df['count_matches'])
    false_neg = sum(df['count_lab_not_found'])
    false_pos = sum(df['count_found_not_lab'])

    precision = true_pos / (true_pos + false_pos) # low number means lots of false positives
    recall = true_pos / (true_pos + false_neg) # low number means lots of false negatives
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    return true_pos, false_neg, false_pos, precision, recall, f1_score

In [24]:
#df_lab_ner_clean

### Process company name matching

In [25]:
labelled_data = False

if labelled_data:
    df_lab_ner_matched = match_companies(df_lab_ner_clean)
    df_lab_ner_matched = check_matches(df_lab_ner_matched)
    true_pos, false_neg, false_pos, precision, recall, f1_score = calculate_performance(df_lab_ner_matched)
    print(true_pos, false_neg, false_pos)
    print(f'precision = {round(precision,2)}')
    print(f'recall = {round(recall,2)}')
    print(f'f1 score = {round(f1_score,2)}')
    df_lab_ner_matched.to_pickle('./data/df_lab_ner_matched.pickle')
    

In [37]:
df_ner_clean.loc[261643:]['org_names']

261643    [& One-offs2, - Network, AFR, AP, Algorithmic ...
261644                           [UTC, the Federal Council]
261645    [Company’s, NYSE, SME, Small/Medium Enterprise...
261646    [Adjusted EBITDA, Business Highlights, Combine...
261647    [Aetna, Associated Press News, CVS Health Corp...
                                ...                        
369042    [ATM, BBC Radio Scotland, C. Hoare & Co., Lloy...
369043    [Artificial Intelligence and Machine Learning,...
369044    [EMV, Jodan Ledford, PAAY, PEP, PR & Editorial...
369045    [Energy\nCorp., Glimpses, Neutron Energy, Inc....
369046    [AJ Bell, British American Tobacco, Connells L...
Name: org_names, Length: 107404, dtype: object

In [28]:
df_ner_matched = pd.read_pickle('./data/df_ner_matched.pickle')

In [30]:
df_ner_matched[df_ner_matched['filtered_names_match'].isna()].iloc[0].name

304403

In [38]:
df_ner_matched.loc[261643:,'org_names'] = df_ner_clean.loc[261643:]['org_names']

In [39]:
df_ner_matched[df_ner_matched['org_names'] == ""]

,author,date,domain,title,url,content,topic_area,org_names,filtered_names,filtered_names_match


In [46]:
df_ner_matched.loc[261643:,['filtered_names','filtered_names_match']] = np.nan

In [47]:
df_ner_matched.loc[261643:]

,author,date,domain,title,url,content,topic_area,org_names,filtered_names,filtered_names_match
261643,NaN,2020-08-05,marketscreener,Deutsche Post : August 2020 Roadshow Presentation,https://www.marketscreener.com/quote/stock/DEU...,INVESTOR PRESENTATION Investor Relations Deuts...,business,"[& One-offs2, - Network, AFR, AP, Algorithmic ...",NaN,NaN
261644,NaN,2020-08-05,marketscreener,Cushman & Wakefield : COVID-19 Impacts on Swit...,https://www.marketscreener.com/quote/stock/CUS...,Switzerland has had very few new COVID-19 case...,business,"[UTC, the Federal Council]",NaN,NaN
261645,NaN,2020-08-05,marketscreener,Western Union : Provides Access to COVID-19 St...,https://www.marketscreener.com/quote/stock/WES...,Western Union broadens its range of services f...,business,"[Company’s, NYSE, SME, Small/Medium Enterprise...",NaN,NaN
261646,NaN,2020-08-05,marketscreener,eXp World Holdings Reports Record Second-quart...,https://www.marketscreener.com/quote/stock/EXP...,Achieves Most Profitable Quarter in Company’s ...,business,"[Adjusted EBITDA, Business Highlights, Combine...",NaN,NaN
261647,NaN,2020-08-05,marketscreener,"CVS Health : posts strong 2Q, with virus delay...",https://www.marketscreener.com/quote/stock/CVS...,CVS Health reported surprisingly strong second...,business,"[Aetna, Associated Press News, CVS Health Corp...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
369042,Polly Harrison,2020-12-31,thefintechtimes,A Human Touch Will Be a Competitive Edge After...,https://thefintechtimes.com/53867-2/,Niels Pedersen is a Chartered Accountant and S...,finance,"[ATM, BBC Radio Scotland, C. Hoare & Co., Lloy...",NaN,NaN
369043,NaN,2020-12-31,marketscreener,Datametrex AI : Announces Deploying NexaSecuri...,https://www.marketscreener.com/quote/stock/DAT...,Datametrex AI Limited (TSXV: DM) (FSE: D4G) (O...,business,"[Artificial Intelligence and Machine Learning,...",NaN,NaN
369044,Polly Harrison,2020-12-31,thefintechtimes,"US Payments: Smart Pension, Episode Six, PAAY ...",https://thefintechtimes.com/us-payments-smart-...,"This December, The Fintech Times is asking ind...",finance,"[EMV, Jodan Ledford, PAAY, PEP, PR & Editorial...",NaN,NaN
369045,NaN,2020-12-31,marketscreener,"WESTWATER RESOURCES, INC. : Entry into a Mater...",https://www.marketscreener.com/quote/stock/WES...,Item 1.01Entry into a Material Definitive Agre...,business,"[Energy\nCorp., Glimpses, Neutron Energy, Inc....",NaN,NaN


In [48]:
#df_ner_matched.to_pickle('./data/df_ner_matched_210913.pickle')

In [30]:
full_data = False
partial_data = False
articles_to_process = 1900
first_pass = False
# total processing will take 9 days so processed in stages

if full_data:
    df_ner_matched = match_companies(df_ner_clean, file_name='df_ner_matched')
    df_ner_matched.to_pickle('./data/df_ner_matched.pickle')  
elif partial_data:
    if first_pass:
        df_ner_matched = df_ner_clean.copy() # only or first time
        start_index = 0
    else:
        df_ner_matched = pd.read_pickle('./data/df_ner_matched.pickle')
        start_index = df_ner_matched[df_ner_matched['filtered_names_match'].isna()].iloc[0].name
        print(f'start index: {start_index}')
    end_index = start_index + articles_to_process
    df_ner_matched_subset = df_ner_matched[start_index:end_index]
    df_ner_matched_subset = match_companies(df_ner_matched_subset, file_name='df_ner_matched')
    df_ner_matched_new = df_ner_matched[:start_index].append(df_ner_matched_subset).append(df_ner_matched[end_index:])
    df_ner_matched_new.to_pickle('./data/df_ner_matched.pickle')
else:
    df_ner_matched = pd.read_pickle('./data/df_ner_matched.pickle')

0it [00:00, ?it/s]

start index: 1100


34it [01:29,  2.79s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: 'Services']
80it [03:12,  1.82s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: 'American']
93it [03:50,  3.18s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: 'Energy']
103it [04:07,  1.41s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: 'American']
116it [04:28,  1.33s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: 'Bank']
133it [05:10,  1.76s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: 'Group']
134it [05:10,  1.49s/it]WARNING:root:Applied processor reduces input query to empty string, all comparisons will have score 0. [Qu

In [94]:
print(df_ner_matched[1188:1200].iloc[0]['filtered_names'])
print(df_ner_matched[1188:1200].iloc[0]['filtered_names_match'])

['Advanced Micro Devices', 'Apple', 'Dow', "McDonald's", 'General Electric', 'AT&T', 'Boeing', 'Starbucks', 'Penn National Gaming', 'Penn National', 'EBay', 'Xilinx', 'L Brands']
['Xilinx, Inc.', 'Starbucks Corporation', "McDonald's Corporation", 'AT&T Inc.', 'General Electric Company', 'Ignite International Brands, Ltd.', 'Penn National Gaming, Inc.', 'Dow Inc.', 'The Boeing Company', 'Advanced Micro Devices, Inc.', 'eBay Inc.', 'Apple Inc.']


In [2]:
df_ner_matched = pd.read_pickle('./data/df_ner_matched.pickle')
df_ner_clean = pd.read_pickle('./data/df_ner_210717.pickle')

In [30]:
df_ner_clean

,author,date,domain,title,url,content,topic_area,org_names
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,"[BA, Boeing, CAT, CMI, Caterpillar, Cummins In..."
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business,"[ADP, AKA, CTAS, Cintas, Cloud-Based Services ..."
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business,"[Credit Suisse, FactSet, Ford, NYSE, Tesla, We..."
3,Roberto Torres,2020-01-03,ciodive,"On the road to AI adoption, execs grapple with...",https://www.ciodive.com/news/ai-adoption-execs...,CIOs kicked off 2019 with AI as an item to wat...,tech,"[AI, CIO, Gartner, Gartner's, H&M, IBM, IBM Data]"
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer,"[11:30pm PST, BAFTA, CDC, Chanel, Instagram, L..."
...,...,...,...,...,...,...,...,...
369042,Polly Harrison,2020-12-31,thefintechtimes,A Human Touch Will Be a Competitive Edge After...,https://thefintechtimes.com/53867-2/,Niels Pedersen is a Chartered Accountant and S...,finance,
369043,NaN,2020-12-31,marketscreener,Datametrex AI : Announces Deploying NexaSecuri...,https://www.marketscreener.com/quote/stock/DAT...,Datametrex AI Limited (TSXV: DM) (FSE: D4G) (O...,business,
369044,Polly Harrison,2020-12-31,thefintechtimes,"US Payments: Smart Pension, Episode Six, PAAY ...",https://thefintechtimes.com/us-payments-smart-...,"This December, The Fintech Times is asking ind...",finance,
369045,NaN,2020-12-31,marketscreener,"WESTWATER RESOURCES, INC. : Entry into a Mater...",https://www.marketscreener.com/quote/stock/WES...,Item 1.01Entry into a Material Definitive Agre...,business,


In [4]:
df_ner_matched#.head(1)

,author,date,domain,title,url,content,topic_area,org_names,filtered_names,filtered_names_match
0,Thomas Hughes,2020-01-02,marketbeat,Three Industrial Giants You Should Own In 2020,https://www.marketbeat.com/originals/three-ind...,With the end of the year just around the corne...,business,"[BA, Boeing, CAT, CMI, Caterpillar, Cummins In...","[Boeing, Caterpillar, Cummins Inc, Deere & Com...","[Caterpillar Inc., Cummins Inc., Deere & Compa..."
1,Thomas Hughes,2020-01-03,marketbeat,Labor Stocks Are Going To Break Out In 2020,https://www.marketbeat.com/originals/labor-sto...,The labor markets were one of the most closely...,business,"[ADP, AKA, CTAS, Cintas, Cloud-Based Services ...",[Cintas],[Cintas Corporation]
2,Steve Anderson,2020-01-03,marketbeat,"Tesla (TSLA) Breaks Shipment Record, Beats Est...",https://www.marketbeat.com/originals/teal-brea...,"It could be forgiven, that some might think th...",business,"[Credit Suisse, FactSet, Ford, NYSE, Tesla, We...",[Tesla],"[Tesla, Inc.]"
3,Roberto Torres,2020-01-03,ciodive,"On the road to AI adoption, execs grapple with...",https://www.ciodive.com/news/ai-adoption-execs...,CIOs kicked off 2019 with AI as an item to wat...,tech,"[AI, CIO, Gartner, Gartner's, H&M, IBM, IBM Data]",[Gartner],"[Gartner, Inc.]"
4,Alden Wicker,2020-01-06,instyle,Red Carpet Sustainability After Coronavirus Sh...,https://www.instyle.com/fashion/red-carpet-cor...,When the coronavirus pandemic is over and life...,consumer,"[11:30pm PST, BAFTA, CDC, Chanel, Instagram, L...",[],[]
...,...,...,...,...,...,...,...,...,...,...
369042,Polly Harrison,2020-12-31,thefintechtimes,A Human Touch Will Be a Competitive Edge After...,https://thefintechtimes.com/53867-2/,Niels Pedersen is a Chartered Accountant and S...,finance,,NaN,NaN
369043,NaN,2020-12-31,marketscreener,Datametrex AI : Announces Deploying NexaSecuri...,https://www.marketscreener.com/quote/stock/DAT...,Datametrex AI Limited (TSXV: DM) (FSE: D4G) (O...,business,,NaN,NaN
369044,Polly Harrison,2020-12-31,thefintechtimes,"US Payments: Smart Pension, Episode Six, PAAY ...",https://thefintechtimes.com/us-payments-smart-...,"This December, The Fintech Times is asking ind...",finance,,NaN,NaN
369045,NaN,2020-12-31,marketscreener,"WESTWATER RESOURCES, INC. : Entry into a Mater...",https://www.marketscreener.com/quote/stock/WES...,Item 1.01Entry into a Material Definitive Agre...,business,,NaN,NaN


In [5]:
df_ner_matched[df_ner_matched['filtered_names_match'].isna()].iloc[0].name#.get_loc(12301)

304403

In [6]:
df_ner_matched.index.get_loc(304403)

304401

In [21]:
df_ner_matched[df_ner_matched['filtered_names_match'].isna()].iloc[0].index

Index(['author', 'date', 'domain', 'title', 'url', 'content', 'topic_area',
       'org_names', 'filtered_names', 'filtered_names_match'],
      dtype='object')

In [29]:
df_ner_clean[261631:261661]

,author,date,domain,title,url,content,topic_area,org_names
261633,NaN,2020-08-05,marketscreener,CoreCivic : Corporate Structure Presentation,https://www.marketscreener.com/quote/stock/COR...,C-Corp Conversion & Capital Allocation Strateg...,business,"[ADJUSTED EBITDA, ADOC ALABAMA FACILITY OVERVI..."
261634,NaN,2020-08-05,marketscreener,ALCONIX : Summary of Consolidated Financial Re...,https://www.marketscreener.com/quote/stock/ALC...,"August 6, 2020 Summary of Consolidated Financi...",business,"[AI, ALCONIX, Additional Information Effect of..."
261635,NaN,2020-08-05,marketscreener,Victory Capital : Reports Second Quarter 2020 ...,https://www.marketscreener.com/quote/stock/VIC...,Second Quarter 2020 Highlights Victory Capital...,business,"[2The Company, AUM, About Victory Capital Vict..."
261636,NaN,2020-08-05,marketscreener,Darling Ingredients Inc. : Reports Second Quar...,https://www.marketscreener.com/quote/stock/DAR...,"IRVING, Texas, Aug. 5, 2020 /PRNewswire/ -- Da...",business,"[ASF, Adjusted EBITDA, BSE, Company, Consolida..."
261637,NaN,2020-08-05,marketscreener,Summit Hotel Properties : Reports Second Quart...,https://www.marketscreener.com/quote/stock/SUM...,"AUSTIN, Texas, Aug. 5, 2020 /PRNewswire/ -- Su...",business,"[91.54 Actual Statistics Rooms, ADR, AFFO, Adj..."
261638,NaN,2020-08-05,marketscreener,Essential Properties Realty Trust : Announces ...,https://www.marketscreener.com/quote/stock/ESS...,- Collected 87% of July Rent - - Second Quarte...,business,"[ABR, AFFO, AFFO The Company, ASC 326, ATM Pro..."
261639,NaN,2020-08-05,marketscreener,CVS Health : Reports Second Quarter Results; D...,https://www.marketscreener.com/quote/stock/CVS...,"WOONSOCKET, R.I., Aug. 5, 2020 /PRNewswire/ --...",business,"[% Prescriptions, 512 Interest, Adjusted EPS, ..."
261640,NaN,2020-08-05,marketscreener,"Energizer Holdings, Inc. : Announces Fiscal 20...",https://www.marketscreener.com/quote/stock/ENE...,"ST. LOUIS, Aug. 5, 2020 /PRNewswire/ -- Energi...",business,"[106.9 99.9 Research, A&P, Acquisition, Adjust..."
261641,NaN,2020-08-05,marketscreener,DBS : Second-Quarter Net Income Fell 22% On Hi...,https://www.marketscreener.com/quote/stock/DBS...,By P.R. Venkat DBS Group Holdings Ltd.'s secon...,business,"[DBS, Factset, P.R. Venkat DBS Group Holdings ..."
261642,NaN,2020-08-05,marketscreener,Cerecor to Present at the 2020 Wedbush PacGrow...,https://www.marketscreener.com/quote/stock/CER...,"ROCKVILLE, Md., Aug. 05, 2020 (GLOBE NEWSWIR...",business,"[About Cerecor Cerecor, Adult Onset Stills Dis..."


In [25]:
df_ner_matched[261631:261661]

,author,date,domain,title,url,content,topic_area,org_names,filtered_names,filtered_names_match
261633,NaN,2020-08-05,marketscreener,CoreCivic : Corporate Structure Presentation,https://www.marketscreener.com/quote/stock/COR...,C-Corp Conversion & Capital Allocation Strateg...,business,"[ADJUSTED EBITDA, ADOC ALABAMA FACILITY OVERVI...","[Company, Flow, TTM]","[American International Holdings Corp., Flow C..."
261634,NaN,2020-08-05,marketscreener,ALCONIX : Summary of Consolidated Financial Re...,https://www.marketscreener.com/quote/stock/ALC...,"August 6, 2020 Summary of Consolidated Financi...",business,"[AI, ALCONIX, Additional Information Effect of...","[IoT, Property, Treasury]","[First Property Group plc, IOT Group Limited, ..."
261635,NaN,2020-08-05,marketscreener,Victory Capital : Reports Second Quarter 2020 ...,https://www.marketscreener.com/quote/stock/VIC...,Second Quarter 2020 Highlights Victory Capital...,business,"[2The Company, AUM, About Victory Capital Vict...","[NASDAQ, Victory Capital, Victory Capital Hold...","[Nasdaq, Inc., Victory Capital Corp., Victory ..."
261636,NaN,2020-08-05,marketscreener,Darling Ingredients Inc. : Reports Second Quar...,https://www.marketscreener.com/quote/stock/DAR...,"IRVING, Texas, Aug. 5, 2020 /PRNewswire/ -- Da...",business,"[ASF, Adjusted EBITDA, BSE, Company, Consolida...","[ASF, Company, Darling Darling Ingredients Inc...","[ASF Group Limited, American International Hol..."
261637,NaN,2020-08-05,marketscreener,Summit Hotel Properties : Reports Second Quart...,https://www.marketscreener.com/quote/stock/SUM...,"AUSTIN, Texas, Aug. 5, 2020 /PRNewswire/ -- Su...",business,"[91.54 Actual Statistics Rooms, ADR, AFFO, Adj...","[Company, Summit Hotel Properties, Summit Hote...","[American International Holdings Corp., Summit..."
261638,NaN,2020-08-05,marketscreener,Essential Properties Realty Trust : Announces ...,https://www.marketscreener.com/quote/stock/ESS...,- Collected 87% of July Rent - - Second Quarte...,business,"[ABR, AFFO, AFFO The Company, ASC 326, ATM Pro...","[Company, Essential Properties]","[American International Holdings Corp., Essent..."
261639,NaN,2020-08-05,marketscreener,CVS Health : Reports Second Quarter Results; D...,https://www.marketscreener.com/quote/stock/CVS...,"WOONSOCKET, R.I., Aug. 5, 2020 /PRNewswire/ --...",business,"[% Prescriptions, 512 Interest, Adjusted EPS, ...","[CVS, CVS HEALTH, CVS Health, CVS Health Corpo...","[American International Holdings Corp., CVS Gr..."
261640,NaN,2020-08-05,marketscreener,"Energizer Holdings, Inc. : Announces Fiscal 20...",https://www.marketscreener.com/quote/stock/ENE...,"ST. LOUIS, Aug. 5, 2020 /PRNewswire/ -- Energi...",business,"[106.9 99.9 Research, A&P, Acquisition, Adjust...","[ENERGIZER HOLDINGS, Energizer Holdings, Share...","[Energizer Holdings, Inc., Share Plc, Treasury..."
261641,NaN,2020-08-05,marketscreener,DBS : Second-Quarter Net Income Fell 22% On Hi...,https://www.marketscreener.com/quote/stock/DBS...,By P.R. Venkat DBS Group Holdings Ltd.'s secon...,business,"[DBS, Factset, P.R. Venkat DBS Group Holdings ...",[],[]
261642,NaN,2020-08-05,marketscreener,Cerecor to Present at the 2020 Wedbush PacGrow...,https://www.marketscreener.com/quote/stock/CER...,"ROCKVILLE, Md., Aug. 05, 2020 (GLOBE NEWSWIR...",business,"[About Cerecor Cerecor, Adult Onset Stills Dis...","[Cerecor, Cerecor Inc., NASDAQ]","[Cerecor Inc., Nasdaq, Inc.]"


In [102]:
print(df_ner_clean.iloc[6000][['content']].values[0])

Half of Samsung's smartphones are now made in Vietnam, where the coronavirus that has crippled the China operations of Apple and many other firms has so far had only a limited impact on its production. Apple said on Monday it would not meet its revenue guidance for the March quarter due to the coronavirus impact on both production and sales in China, where most iPhones are made. Chinese smartphone maker Xiaomi Corp last week also flagged a hit to its March quarter sales. Huawei, another major Samsung rival, has not announced any production problems, but analysts expect it will also be hit hard due to its heavy reliance on Chinese manufacturing and parts. Many Chinese and foreign firms have begun to re-open China factories that were idled for weeks, but shortages of workers and other problems have in many cases kept output to a minimum. Samsung has also largely ceded the China market to its rivals in recent years, meaning it won't suffer from the store closures and drop in demand that i

In [100]:
print(df_ner_clean.iloc[6000][['filtered_names_match']].values[0])

{'Samsung Electronics CO L', 'apple computer, inc.', 'sealand capital galaxy limited', 'apple hospitality reit inc', 'Samsung Electronics Co L', 'apple inc', 'apple inc.', 'samsung electronics co l'}
